# 복지시설 데이터 동별 매칭

⇒ 돌봄시설 API

규호: 공원, 도서관, 체육시설, 대중교통

하은: 경찰범죄통계, 5대 범죄 구별 범죄율, 주차, 57번

재영: 초/ 중/고등/대학교, 도로, (각 초등학교가 커버하는 범위?), 56번, 31번

동명: 상권, 학원가, 공장, 병원, 69번, 58번

소연: 건폐율, 전용면적, 각 행정동 안에 주택개수(코드)

⇒ 행정동별로 개수, 평균

⇒ X, Y 좌표 (위도, 경도)

⇒ 동별로 없으면? 구별로도

⇒ 개수, 면적(규모)

In [ ]:
# 필요한 라이브러리 임포트&설치
!pip install dbfread
!pip install haversine
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

서울시 전체에 분포한

공원, 도서관, 대중교통(지하철/버스), 체육시설, +미술관박물관

을 동별로 몇개씩 있는지 분배한 데이터프레임을 만들면 되는데,

동별이 안되면 구별로 하면 됨.

위도경도 따져주고

가능하면 부가정보도(넓이라던가 수용인원이라던가 연식이라던가)

1. 일단 주소/위도경도만 남기고 다 없애자(흔적은 columns해서 남길것)

2. re findall로 동만 추출한다거나, 구 추출해서 매칭해주기(새로운 열)

3. 데이터 적은거로 구별 중심으로 해서 가까운곳 매칭하는것(지하철/버스) 해서 옆에 ~동 이렇게 하던지 동옆에 ~개 이렇게 하던지.


In [ ]:
###필요한 데이터 불러오기
#대중교통-버스정류소
bus = pd.read_excel(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시버스정류소좌표데이터(2020.03.06).xlsx', encoding='cp949')
#대중교통-지하철
sub = pd.read_excel(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울교통공사 역별 주소 현황 및 전화번호(구주소).xlsx', encoding='cp949')
#공원
park = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시 주요 공원현황.csv', encoding='cp949')
#박물관
museum = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울특별시_박물관미술관정보_20180821.csv', encoding='cp949')
#도서관
lib = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\전국도서관표준데이터.csv', encoding='cp949')
#체육시설-하은이 기다려야함
# sports = pd.read_csv()

#구 좌표
gu_loc = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시 행정구역 시군구 정보 (좌표계_ WGS1984).csv', encoding='cp949')
#동 좌표
dong_loc= pd.read_excel(r'C:\Users\이규호\NLP\github\seoul_contest\data\행정_법정동 중심좌표.xlsx', encoding='cp949')
#행정동목록
dong = pd.read_excel(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시_행정동_법정동_20200814.xlsx', encoding='cp949')


출처:

버스:https://data.seoul.go.kr/dataList/OA-15067/S/1/datasetView.do

공원: http://data.seoul.go.kr/dataList/OA-394/S/1/datasetView.do;jsessionid=C03761A8C8B90B16BD4AD7EAD8E06367.new_portal-svr-21

미술/박물관:http://data.seoul.go.kr/dataList/OA-15272/F/1/datasetView.do

도서관:https://www.data.go.kr/data/15013109/standard.do

지하철:http://www.seoulmetro.co.kr/kr/board.do?menuIdx=551&bbsIdx=2208459

구좌표:https://data.seoul.go.kr/dataList/OA-11677/S/1/datasetView.do

동좌표:https://torrms.tistory.com/55

행정동목록: https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardList.do?bbsId=BBSMSTR_000000000052

# 원본 데이터 오픈

In [ ]:
bus

In [ ]:
sub

In [ ]:
park

In [ ]:
museum

In [ ]:
lib

In [ ]:
# sports

In [ ]:
gu_loc

In [ ]:
dong_loc = dong_loc[dong_loc['코드종류']=='B']#법정동 코드 불러오기. 이유는 행정동코드는 위도경도를 중복시켜놔서 1동과 2동을 같은 위치로 보는 경우가 많음. 425개 살려야하는데, 267개밖에 살리지 못한다. 법정동으로 매칭시키고 다시 행정동에 배정하기.
dong_loc = dong_loc[dong_loc['시도']=='서울특별시']#서울만 사용
dong_loc

In [ ]:
dong

# 각 데이터 컬럼 보기

In [ ]:
bus.columns

In [ ]:
sub.columns

In [ ]:
park.columns

In [1055]:
museum.columns

Index(['시설명', '박물관미술관구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '운영기관전화번호',
       '운영기관명', '운영홈페이지', '편의시설정보', '평일관람시작시각', '평일관람종료시각', '공휴일관람시작시각',
       '공휴일관람종료시각', '휴관정보', '어른관람료', '청소년관람료', '어린이관람료', '관람료기타정보', '박물관미술관소개',
       '교통안내정보', '관리기관전화번호', '관리기관명', '데이터기준일자'],
      dtype='object')

In [1056]:
lib.columns

Index(['도서관명', '시도명', '시군구명', '도서관유형', '휴관일', '평일운영시작시각', '평일운영종료시각',
       '토요일운영시작시각', '토요일운영종료시각', '공휴일운영시작시각', '공휴일운영종료시각', '열람좌석수', '자료수(도서)',
       '자료수(연속간행물)', '자료수(비도서)', '대출가능권수', '대출가능일수', '소재지도로명주소', '운영기관명',
       '도서관전화번호', '부지면적', '건물면적', '홈페이지주소', '위도', '경도', '데이터기준일자', '제공기관코드',
       '제공기관명'],
      dtype='object')

In [1057]:
gu_loc.columns

Index(['순번', '시군구코드', '시군구명_한글', '시군구명_영문', 'ESRI_PK', '위도', '경도'], dtype='object')

In [1058]:
dong_loc.columns

Index(['코드', '시도', '시군구', '읍면동', '하위', '위도', '경도', '코드종류'], dtype='object')

In [1059]:
dong.columns

Index(['행정동코드', '시도명', '시군구명', '읍면동명', '법정동코드', '동리명', '생성일자', '말소일자'], dtype='object')

# 필요한 열만 남기는 전처리

우선, 시설의 이름과 주소, 위도경도가 필요하다. 연식/넓이 등은 나중에 컬럼보고 돌아와서 추가할 것

완성되면 xx_df로 이름짓기

In [1060]:
bus.head()

,표준ID,ARS-ID,정류장명,X좌표,Y좌표,비고
0,100000001,1001,종로2가사거리,126.987750,37.569765,NaN
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,NaN
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,NaN
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,NaN
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,NaN


In [1061]:
bus_df = bus[['정류장명','Y좌표','X좌표','표준ID','ARS-ID']]
bus_df.columns = ['정류장명','위도','경도','표준ID','ARS-ID']

In [1062]:
sub.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,연번,역명,호선,구,동,상세주소,전화번호
1,1,서울,1,중구,봉래동,서울특별시 중구 봉래동2가 122 서울역(1호선),6110-1331
2,2,시청,1,중구,정동,서울특별시 중구 정동 5-5 시청역(1호선),6110-1321
3,3,종각,1,종로구,종로1가,서울특별시 종로구 종로1가 54 종각지하철역사,6110-1311
4,4,종로3가,1,종로구,종로3가,서울특별시 종로3가 10-5 1호선 종로3가역(1호선),6110-1301


In [1063]:
# sub_df = sub[[]]

In [1064]:
park.head()

,공원번호,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,3,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120,448852.675,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,4,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267,437163.981,127.019847,37.426449,http://grandpark.seoul.go.kr/
3,5,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432,449290.726,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...
4,6,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070,451598.831,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...


In [1065]:
park_df = park[['공원명','Y좌표(WGS84)','X좌표(WGS84)','지역','공원주소']]
park_df.columns = ['공원명','위도','경도','지역','공원주소']

In [1066]:
museum.head()

,시설명,박물관미술관구분,소재지도로명주소,소재지지번주소,위도,경도,운영기관전화번호,운영기관명,운영홈페이지,편의시설정보,...,휴관정보,어른관람료,청소년관람료,어린이관람료,관람료기타정보,박물관미술관소개,교통안내정보,관리기관전화번호,관리기관명,데이터기준일자
0,둘리뮤지엄,공립,서울특별시 도봉구 시루봉로 1길 6,NaN,37.652432,127.027640,02-990-2200,도봉구,doolymuseum.or.kr,NaN,...,홈페이지참고,5000,5000,5000,NaN,NaN,NaN,02-990-2200,도봉구,2017-12-31
1,서대문자연사박물관,공립,서울특별시 서대문구 연희로 32길 51,NaN,37.576753,126.937858,02-330-8899,서대문구,http://namu.sdm.go.kr,NaN,...,홈페이지참고,6000,3000,2000,NaN,NaN,NaN,02-330-8899,서대문구,2017-12-31
2,서대문형무소역사관,공립,서울특별시 서대문구 통일로251,NaN,37.572442,126.959541,02-360-8590,서대문구,www.sscmc.or.kr,NaN,...,홈페이지참고,3000,1500,1000,NaN,NaN,NaN,02-360-8590,서대문구,2017-12-31
3,서울교육박물관,공립,서울특별시 종로구 북촌로 5길 48,NaN,37.581031,126.983694,02-2011-5780,서울특별시특별시교육청,http://edumuseum.sen.go.kr,NaN,...,홈페이지참고,0,0,0,NaN,NaN,NaN,02-2011-5780,서울특별시특별시교육청,2017-12-31
4,서울상상나라,공립,서울특별시 광진구 능동로 216,NaN,37.549458,127.086261,02-6450-9500,서울특별시특별시,www.seoulchildrensmuseum.org,NaN,...,홈페이지참고,4000,4000,4000,NaN,NaN,NaN,02-6450-9500,서울특별시특별시,2017-12-31


In [1067]:
museum.columns

Index(['시설명', '박물관미술관구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '운영기관전화번호',
       '운영기관명', '운영홈페이지', '편의시설정보', '평일관람시작시각', '평일관람종료시각', '공휴일관람시작시각',
       '공휴일관람종료시각', '휴관정보', '어른관람료', '청소년관람료', '어린이관람료', '관람료기타정보', '박물관미술관소개',
       '교통안내정보', '관리기관전화번호', '관리기관명', '데이터기준일자'],
      dtype='object')

In [1068]:
museum_df = museum[['시설명','위도','경도','소재지도로명주소','운영기관명','관리기관명']]

In [1069]:
lib.head()

,도서관명,시도명,시군구명,도서관유형,휴관일,평일운영시작시각,평일운영종료시각,토요일운영시작시각,토요일운영종료시각,공휴일운영시작시각,...,운영기관명,도서관전화번호,부지면적,건물면적,홈페이지주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,덕산아내프리미엄아파트 작은도서관,경상남도,거제시,작은도서관,"매주월요일, 공휴일",16:00,22:00,16:00,22:00,16:00,...,경상남도 거제시 덕산아내프리미엄1차아파트입주자대표회,055-681-9766,31675.0,142.0,NaN,34.866433,128.684936,2020-01-30,5370000,경상남도 거제시
1,강하작은도서관,경기도,양평군,작은도서관,"매주일,월/공휴일",09:00,18:00,09:00,18:00,00:00,...,경기도 양평군청,031-771-4302,634.0,179.0,http://www.yplib.go.kr,37.496177,127.411217,2020-03-06,4170000,경기도 양평군
2,강상작은도서관,경기도,양평군,작은도서관,"매주일,월/공휴일",09:00,18:00,09:00,18:00,00:00,...,경기도 양평군청,031-775-7483,3927.0,140.0,http://www.yplib.go.kr,37.480695,127.478067,2020-03-06,4170000,경기도 양평군
3,양동도서관,경기도,양평군,공공도서관,"신정,설,추석연휴",08:00,22:00,08:00,22:00,08:00,...,경기도 양평군청,031-770-2571,3434.0,1314.0,http://www.yplib.go.kr,37.419191,127.754688,2020-03-06,4170000,경기도 양평군
4,지평도서관,경기도,양평군,공공도서관,"신정,설,추석연휴",08:00,22:00,08:00,22:00,08:00,...,경기도 양평군청,031-770-2561,2215.0,1045.0,http://www.yplib.go.kr,37.473719,127.638517,2020-03-06,4170000,경기도 양평군


In [1070]:
lib.columns

Index(['도서관명', '시도명', '시군구명', '도서관유형', '휴관일', '평일운영시작시각', '평일운영종료시각',
       '토요일운영시작시각', '토요일운영종료시각', '공휴일운영시작시각', '공휴일운영종료시각', '열람좌석수', '자료수(도서)',
       '자료수(연속간행물)', '자료수(비도서)', '대출가능권수', '대출가능일수', '소재지도로명주소', '운영기관명',
       '도서관전화번호', '부지면적', '건물면적', '홈페이지주소', '위도', '경도', '데이터기준일자', '제공기관코드',
       '제공기관명'],
      dtype='object')

In [1071]:
lib = lib[lib['시도명']=='서울특별시']
lib_df = lib[['도서관명','위도','경도','시군구명','소재지도로명주소','운영기관명']]

In [1072]:
gu_loc.head()

,순번,시군구코드,시군구명_한글,시군구명_영문,ESRI_PK,위도,경도
0,1,11320,도봉구,Dobong-gu,0,37.665861,127.031767
1,2,11380,은평구,Eunpyeong-gu,1,37.617612,126.922700
2,3,11230,동대문구,Dongdaemun-gu,2,37.583801,127.050700
3,4,11590,동작구,Dongjak-gu,3,37.496504,126.944307
4,5,11545,금천구,Geumcheon-gu,4,37.460097,126.900155


In [1073]:
gu_loc_df = gu_loc[['시군구명_한글','위도','경도']]

In [1074]:
dong_loc.읍면동.unique()

array(['청운동', '신교동', '궁정동', '효자동', '창성동', '통의동', '적선동', '통인동', '누상동',
       '누하동', '옥인동', '체부동', '필운동', '내자동', '사직동', '도렴동', '당주동', '내수동',
       '세종로', '신문로1가', '신문로2가', '청진동', '서린동', '수송동', '중학동', '종로1가', '공평동',
       '관훈동', '견지동', '와룡동', '권농동', '운니동', '익선동', '경운동', '관철동', '인사동',
       '낙원동', '종로2가', '팔판동', '삼청동', '안국동', '소격동', '화동', '사간동', '송현동',
       '가회동', '재동', '계동', '원서동', '훈정동', '묘동', '봉익동', '돈의동', '장사동', '관수동',
       '종로3가', '인의동', '예지동', '원남동', '연지동', '종로4가', '효제동', '종로5가', '종로6가',
       '이화동', '연건동', '충신동', '동숭동', '혜화동', '명륜1가', '명륜2가', '명륜4가', '명륜3가',
       '창신동', '숭인동', '교남동', '평동', '송월동', '홍파동', '교북동', '행촌동', '구기동',
       '평창동', '부암동', '홍지동', '신영동', '무악동', '무교동', '다동', '태평로1가', '을지로1가',
       '을지로2가', '남대문로1가', '삼각동', '수하동', '장교동', '수표동', '소공동', '남창동', '북창동',
       '태평로2가', '남대문로2가', '남대문로3가', '남대문로4가', '남대문로5가', '봉래동1가', '봉래동2가',
       '회현동1가', '회현동2가', '회현동3가', '충무로1가', '충무로2가', '명동1가', '명동2가',
       '남산동1가', '남산동2가', '남산동3가', '저동1가', '충무로4가', '충무로5가', '인현동

In [1075]:
dong_loc_df = dong_loc[['코드','위도','경도']]
dong_loc_df.columns = ['법정동코드','위도','경도']

In [1076]:
dong.head()

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN


In [1077]:
dong = dong[dong['시도명']=='서울특별시']
dong_df = dong[['시군구명','읍면동명','동리명','행정동코드','법정동코드']]

# df들만 모아보기

In [1078]:
bus_df

,정류장명,위도,경도,표준ID,ARS-ID
0,종로2가사거리,37.569765,126.987750,100000001,1001
1,창경궁.서울대학교병원,37.579183,126.996566,100000002,1002
2,명륜3가.성대입구,37.582671,126.998340,100000003,1003
3,종로2가.삼일교,37.568579,126.987613,100000004,1004
4,혜화동로터리.여운형활동터,37.586243,127.001744,100000005,1005
...,...,...,...,...,...
11275,우성아파트,37.550386,127.139339,124000334,25995
11276,우성아파트,37.550643,127.140046,124000333,25996
11277,조일약국,37.533630,127.123596,124000332,25997
11278,성내시장,37.536155,127.125497,124000331,25998


In [1079]:
# sub_df

In [1080]:
park_df

,공원명,위도,경도,지역,공원주소
0,남산도시자연공원,37.550140,126.990377,중구,서울특별시 중구 삼일대로 231(예장동)
1,길동생태공원,37.540394,127.154779,강동구,서울특별시 강동구 천호대로 1291(길동생태공원)
2,서울대공원,37.426449,127.019847,과천시,경기도 과천시 대공원광장로 102
3,서울숲,37.543072,127.041798,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가)
4,월드컵공원,37.571805,126.878907,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원)
...,...,...,...,...,...
127,용마도시자연공원(사가정공원),37.579762,127.095997,중랑구,서울특별시 중랑구 면목7동 산50
128,문화비축기지,37.571718,126.893245,마포구,서울특별시 마포구 증산로 87
129,경춘선숲길,37.627077,127.076482,NaN,노원구 공릉2동 산 82-2
130,율현공원,37.472332,127.115594,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56)


In [1081]:
museum_df

,시설명,위도,경도,소재지도로명주소,운영기관명,관리기관명
0,둘리뮤지엄,37.652432,127.027640,서울특별시 도봉구 시루봉로 1길 6,도봉구,도봉구
1,서대문자연사박물관,37.576753,126.937858,서울특별시 서대문구 연희로 32길 51,서대문구,서대문구
2,서대문형무소역사관,37.572442,126.959541,서울특별시 서대문구 통일로251,서대문구,서대문구
3,서울교육박물관,37.581031,126.983694,서울특별시 종로구 북촌로 5길 48,서울특별시특별시교육청,서울특별시특별시교육청
4,서울상상나라,37.549458,127.086261,서울특별시 광진구 능동로 216,서울특별시특별시,서울특별시특별시
...,...,...,...,...,...,...
154,포스코미술관,37.505831,127.056088,서울특별시 강남구 대치4동 892 포스코센터 내,㈜포스코,㈜포스코
155,학아재미술관,NaN,NaN,서울특별시 종로구 돈화문로 98,김로이,김로이
156,한미사진미술관,37.516608,127.113970,"서울특별시 송파구 위례성대로 14 한미타워 19, 20층",가현문화재단,가현문화재단
157,헬로우뮤지움,37.507358,127.040312,서울특별시 성동구 금호로 72,김효정\n(김이삭),김효정\n(김이삭)


In [1082]:
lib_df

,도서관명,위도,경도,시군구명,소재지도로명주소,운영기관명
103,대방어린이도서관,37.503024,126.925222,동작구,서울특별시 동작구 대방동길 55,서울특별시 동작구청
105,동작어린이도서관,37.509204,126.943434,동작구,서울특별시 동작구 장승배기로16길 98,서울특별시 동작구청
106,동작상도국주도서관,37.506822,126.950996,동작구,서울특별시 동작구 매봉로 37,서울특별시 동작구청
202,넓은들작은도서관,37.511385,126.841418,양천구,서울특별시 양천구 목동중앙남로 16나길 55,서울특별시 양천구청
203,고맙습니다 작은도서관,37.515237,126.854773,양천구,서울특별시 양천구 중앙로 209,서울특별시 양천구청
...,...,...,...,...,...,...
3299,서울특별시교육청도봉도서관(디지털자료실),37.652746,127.012966,서울특별시 도봉구,서울특별시 도봉구 도봉구 삼양로 556(쌍문동),NaN
3300,도봉문화정보도서관(종합자료실),37.644581,127.043883,서울특별시 도봉구,서울특별시 도봉구 도봉구 덕릉로 315 (창동),NaN
3301,도봉문화정보도서관(어린이열람실),37.644581,127.043883,서울특별시 도봉구,서울특별시 도봉구 도봉구 덕릉로 315 (창동),NaN
3302,도봉어린이문화정보도서관(영 유아),37.659007,127.049470,서울특별시 도봉구,서울특별시 도봉구 도봉구 노해로69길 151,NaN


In [1083]:
gu_loc_df

,시군구명_한글,위도,경도
0,도봉구,37.665861,127.031767
1,은평구,37.617612,126.922700
2,동대문구,37.583801,127.050700
3,동작구,37.496504,126.944307
4,금천구,37.460097,126.900155
5,구로구,37.495486,126.858121
6,종로구,37.599100,126.986149
7,강북구,37.646995,127.014716
8,중랑구,37.595379,127.093967
9,강남구,37.495985,127.066409


In [1084]:
dong_loc_df

,법정동코드,위도,경도
3546,1111010100,37.587111,126.969069
3547,1111010200,37.583911,126.968354
3548,1111010300,37.584381,126.971489
3549,1111010400,37.582416,126.971670
3550,1111010500,37.580363,126.972065
...,...,...,...
4008,1174010600,37.531157,127.142891
4009,1174010700,37.552497,127.133010
4010,1174010800,37.531338,127.129174
4011,1174010900,37.543676,127.131748


In [1085]:
dong_df

,시군구명,읍면동명,동리명,행정동코드,법정동코드
0,NaN,NaN,서울특별시,1100000000,1100000000
1,종로구,NaN,종로구,1111000000,1111000000
2,종로구,청운효자동,청운동,1111051500,1111010100
3,종로구,청운효자동,신교동,1111051500,1111010200
4,종로구,청운효자동,궁정동,1111051500,1111010300
...,...,...,...,...,...
763,강동구,성내제2동,성내동,1174065000,1174010800
764,강동구,성내제3동,성내동,1174066000,1174010800
765,강동구,길동,길동,1174068500,1174010500
766,강동구,둔촌제1동,둔촌동,1174069000,1174010600


중간정리: 

데이터는 모두 위도경도 존재.

bus에는 주소정보가 없어서 행정동을 re.findall로 찾아내기가 불가능함.

office는 위도경도/주소정보/구정보가 있지만 '행정복지센터(동사무소)'만 뽑아내려면 전처리 필요

gu_loc은 위도경도가 있지만 '구'단위라서 크게 도움될지는...

dong은 '읍면동'명과 '동리'명의 차이가 있을 듯 한데... 이걸 새주소로 어떻게 바꿔야할지도 고민해야함

즉, 동별 시설을 매칭(라벨링)하려면 3가지 방법중에 하나를 사용해야한다.

1. 주소를 전처리해서 xx동을 뽑아내고, 새로운 열로 라벨링 해주기-도로명주소는 더 어려움

2. 동사무소의 위도경도를 구해서 각각의 데이터들을 가장 가까운 동사무소로 매칭시키고 라벨링 해주기(haversine 활용)-동사무소 위도경도의 무결성 보장불가(빠진 동사무소 존재가능)

3. 2번 방법을 활용해 각각의 데이터들을 구별로 매칭 후, 인구비율등에 따라 분배하고 라벨링 해주기 
-정확도 떨어짐

1번을 활용하려면 bus를 제외하고 주소정보를 전처리 해야한다.

2번을 활용하려면 office를 전처리 해야한다.

3번을 활용하려면 인구비율이 필요하다.

### 1번을 위한 주소정보 전처리

### 공원

In [1086]:
park_df

,공원명,위도,경도,지역,공원주소
0,남산도시자연공원,37.550140,126.990377,중구,서울특별시 중구 삼일대로 231(예장동)
1,길동생태공원,37.540394,127.154779,강동구,서울특별시 강동구 천호대로 1291(길동생태공원)
2,서울대공원,37.426449,127.019847,과천시,경기도 과천시 대공원광장로 102
3,서울숲,37.543072,127.041798,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가)
4,월드컵공원,37.571805,126.878907,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원)
...,...,...,...,...,...
127,용마도시자연공원(사가정공원),37.579762,127.095997,중랑구,서울특별시 중랑구 면목7동 산50
128,문화비축기지,37.571718,126.893245,마포구,서울특별시 마포구 증산로 87
129,경춘선숲길,37.627077,127.076482,NaN,노원구 공릉2동 산 82-2
130,율현공원,37.472332,127.115594,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56)


In [1087]:
# 구 뒤에 나오는 단어를 뽑아서 데이터프레임에 라벨링 해주는 코드. 
# 함수로 발전시키면 좋겠지만, 각각의 데이터프레임이 어떤식으로 주소를 가지고 있을지 몰라 일단 작성후 복사
park_df['동'] = 0
for idx in range(len(park_df.공원주소)):
    if re.findall('구 (.+)', park_df.공원주소.loc[idx]):        

        if re.findall('(.+?) ', re.findall('구 (.+)', park_df.공원주소.loc[idx])[0]):
            park_df['동'].loc[idx] = re.findall('(.+?) ', re.findall('구 (.+)', park_df.공원주소.loc[idx])[0])[0]
            print(re.findall('(.+?) ', re.findall('구 (.+)', park_df.공원주소.loc[idx])[0])[0])
            
        else:
            park_df['동'].loc[idx] = '주의!!' + park_df.공원주소.loc[idx]            
            print('주의!!' + park_df.공원주소.loc[idx])
    else:
        park_df['동'].loc[idx] = '주의!!' + park_df.공원주소.loc[idx]
        print('주의!!' + park_df.공원주소.loc[idx])

삼일대로
천호대로
주의!!경기도 과천시 대공원광장로 102
뚝섬로
하늘공원로
세종로
진관동
영등포동2가
금호동1가
등촌동
성수2가
명일1동
여의공원로
서울시립대로2길
을지로
신길로
여의대방로20길
능동로
올림픽로
서빙고로
주의!!서울특별시 서대문구 통일로251(독립공원)
낙산길
매헌로
관악로
효창원로
독산4동
북악산로
오류동
새문안로
사당2동
신정동
고척로45길
목동동로
허준로5길
성산로7길
가양동
사당동
방화동
여의대방로
답십리로
광평로10길
종로
신정동
마천동
도곡동
도산대로45길
화랑로18가길
창동
고덕2동
대학로8길
의사당대로
장지동
상일동
금낭화로
올림픽로
한천로43길
홍제동
염창동
통일로
오금로
번동
구산동
개봉동
삼성동
올림픽로
와룡공원길
묵동
부암동
은평구
양화진길
불광동
한글비석로12길
면목동
노량진로
목동
사직로9길
내발산동
사당동
북촌로
옥인동
성대로6사길
동남로
쌍문동
상계동
잠실로
선유로
손기정로
돈암동
화곡동
마들로
월계로
남부순환로64길
송림길
우이동
독산동
시흥대로38길
독산동
시루봉로6길
창동
용두동
상암동
창전동
장터1길
옥수동
문정동
바우뫼로12길
세종대로
대학동
창동
연동로
구파발동
원지동
개화동
광장동
내곡동
염곡동
우면동
진관동
신사동
통일로
수서동
청담동
둔촌동
통일로
망우동
숭인동
면목동
면목7동
증산로
공릉2동
율현동
주의!!서울로7017


<ipython-input-1087-37d76bd828fb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park_df['동'] = 0
C:\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-1087-37d76bd828fb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park_df['동'].loc[idx] = re.findall('(.+?) ', re.fi

In [1088]:
park_df.tail(50)

,공원명,위도,경도,지역,공원주소,동
82,쌍문근린공원,37.653146,127.027132,도봉구,서울특별시 도봉구 쌍문동 산80,쌍문동
83,수락산도시자연공원,37.699232,127.081363,노원구,서울특별시 노원구 상계동 산3-1,상계동
84,송파나루근린공원(석촌호수),37.506933,127.098300,송파구,서울특별시 송파구 잠실로 148 (잠실동),잠실로
85,선유도근린공원,37.543418,126.900140,영등포구,서울특별시 영등포구 선유로 343 (당산동 1) 선유도공원관리사무소,선유로
86,손기정체육공원,37.556019,126.965270,중구,서울특별시 중구 손기정로 101,손기정로
87,성북근린공원,37.594851,127.007863,성북구,서울특별시 성북구 돈암동 616-1055,돈암동
88,봉제산공원,37.542198,126.851784,강서구,서울특별시 강서구 화곡동 산41-6,화곡동
89,서울창포원,37.689585,127.047938,도봉구,서울특별시 도봉구 마들로 916(서울창포원),마들로
90,북서울꿈의숲,37.623335,127.040734,강북구,서울특별시 강북구 월계로 173,월계로
91,서서울호수공원,37.527839,126.830479,양천구,서울특별시 양천구 남부순환로64길 20 (신월동),남부순환로64길


# 주의!붙은 애들 까보기+전체적으로 보기

In [1089]:
for idx in range(len(park_df.공원주소)):
    print(park_df.공원주소.loc[idx])

서울특별시 중구 삼일대로 231(예장동)
서울특별시 강동구 천호대로 1291(길동생태공원) 
경기도 과천시 대공원광장로 102
서울특별시 성동구 뚝섬로 273 (성수동1가) 
서울특별시 마포구 하늘공원로 84(월드컵공원)
서울특별시 종로구 세종로 76-2
서울특별시 은평구 진관동 393
서울특별시 영등포구 영등포동2가 222
서울특별시 성동구 금호동1가 산1-72
서울특별시 강서구 등촌동 630-14
서울특별시 성동구 성수2가 3동 302-25
서울특별시 강동구 명일1동 288
서울특별시 영등포구 여의공원로 68 (여의도동)
서울특별시 동대문구 서울시립대로2길 59(간데메공원)
서울특별시 중구 을지로 227
서울특별시 영등포구 신길로 275
서울특별시 동작구 여의대방로20길 33(보라매공원)
서울특별시 광진구 능동로 216(어린이대공원)
서울특별시 강동구 올림픽로 702(천호공원)
서울특별시 용산구 서빙고로 185
서울특별시 서대문구 통일로251(독립공원)
서울특별시 종로구 낙산길 41 (우) 03085
서울특별시 서초구 매헌로 99 (양재동)
서울특별시 관악구 관악로 1(관악산공원)
서울특별시 용산구 효창원로 177-18
서울특별시 금천구 독산4동 371?375 373일대
서울특별시 성북구 북악산로 949-73(돈암동 6-1)
서울특별시 구로구 오류동 산13-1
서울특별시 종로구 새문안로 55(신문로2가 2-1)
서울특별시 동작구 사당2동 산41-2
서울특별시 양천구 신정동 산44-2일대
서울특별시 구로구 고척로45길 39(고척공원)
서울특별시 양천구 목동동로 363 
서울특별시 강서구 허준로5길 42
서울특별시 서대문구 성산로7길 19-24(궁동공원) 
서울특별시 강서구 가양동 산4-7
서울특별시 동작구 사당동 산32
서울특별시 강서구 방화동 산110
서울특별시 동작구 여의대방로 15-45(여의대방로36길 71)
서울특별시 동대문구 답십리로 209(답십리공원) 
서울특별시 강남구 광평로10길 30-71(대모산공원) 
서울특별시 종로구 종로 99
서울특별시 

특이한 애들

주소를 2가지 버전으로 적어둔곳이거나(서울시 oo로/서울시oo동), 

여러곳에 걸쳐 있는곳이거나(서울시 oo동, oo동, oo동), 

동 뒤에 별다른 정보가 없는 경우(서울시 oo구 oo동)-1개, 

서울이 아닌경우(서울대공원).

더 정확히 하려면 추가 전처리 필요할듯. 특히 여러곳에 펼쳐져있는 공원의 경우...

### 미술관/박물관

In [1090]:
museum_df['동'] = 0
for idx in range(len(museum_df.소재지도로명주소)):
    if re.findall('구 (.+)', museum_df.소재지도로명주소.loc[idx]):        

        if re.findall('(.+?) ', re.findall('구 (.+)', museum_df.소재지도로명주소.loc[idx])[0]):
            museum_df['동'].loc[idx] = re.findall('(.+?) ', re.findall('구 (.+)', museum_df.소재지도로명주소.loc[idx])[0])[0]
            print(re.findall('(.+?) ', re.findall('구 (.+)', museum_df.소재지도로명주소.loc[idx])[0])[0])
            
        else:
            museum_df['동'].loc[idx] = '주의!!' + museum_df.소재지도로명주소.loc[idx]            
            print('주의!!' + museum_df.소재지도로명주소.loc[idx])
    else:
        museum_df['동'].loc[idx] = '주의!!' + museum_df.소재지도로명주소.loc[idx]
        print('주의!!' + museum_df.소재지도로명주소.loc[idx])

시루봉로
연희로
주의!!서울특별시 서대문구 통일로251
북촌로
능동로
주의!!서울특별시 동대문구 약령중앙로26
새문안로
성북로
손기정로
왕십리로
연서로50길
창덕궁
청계천로
위례성대로
허준로
북촌로
삼성로
충정로
주의!!서울특별시 서초구 바우뫼로7길111
와우산로
화곡로
홍지문1길
동숭길
새문안로
불광로
도산대로
돈화문로
올림픽로
올림픽로
북촌로6길
인사동
인사동길
우이동
서소문로11길
임정로
인사동
북촌로11길
북촌로
주의!!서울특별시 종로구 북촌로5나길90(삼청동)
우정국로
아리수로61길
주의!!서울특별시 종로구 비봉길2-2
주의!!서울특별시 명동길 74-2
주의!!서울특별시 을지로 23
대학로
올림픽로
주의!!서울특별시 테헤란로 528(대치동 945-5)
북촌로
이화장길
세검정로
세종대로
63로
소월로
평창30길
대사관로
소공로
신촌로129
창의문로
신촌로
정동길
서초중앙로2길
효령로
월드컵북로11길
이태원로
장충단로
성균관로
퇴계로
돈화문로
주의!!서울특별시 언주로 827 코리아나아트센타
주의!!서울특별시 밤고개로14길 13-51
오류로8리길
주의!!서울특별시 청담동 84-9
대사관로
올림픽로35길
세종대로
율곡로10길,
남대문로
여의대방로
토정로
동호로
율곡로
혜화로
한남대로
성북로
주의!!서울특별시 남부순환로 152길 53
주의!!서울특별시 도산대로317
평창8길
능동로
주의!!서울특별시 동대문구 경희대로26
경희대로
경희대로
안암로
주의!!서울특별시 정릉로 77
삼양로
필동로1길
화랑로
평창문화로
대학로
관악로
서울특별시립대로
화랑로
성균관로
보문로
도봉로
도봉로
능동로
청파로
청파로
주의!!서울특별시 동작구 상도로369
연세로
이화여대길
이화여대길
주의!!서울특별시 성동구 왕십리로222
와우산로
주의!!서울특별시 화랑로 13길 60 동덕여자대학교
양천로4길
덕수궁길
남부순환로
성북로
옥인1길
공릉로
관악로
청파로
초안산로
살곶이길
남부순환로
63로
창경궁로35길
선릉로
우정국로
효창원로70길
삼청로
자하문로4길
일원동
율곡로
이태원로
평창31길


<ipython-input-1090-46e1bccb0e9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  museum_df['동'] = 0
C:\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-1090-46e1bccb0e9a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  museum_df['동'].loc[idx] = re.findall('(.+?) ', r

In [1091]:
museum_df

,시설명,위도,경도,소재지도로명주소,운영기관명,관리기관명,동
0,둘리뮤지엄,37.652432,127.027640,서울특별시 도봉구 시루봉로 1길 6,도봉구,도봉구,시루봉로
1,서대문자연사박물관,37.576753,126.937858,서울특별시 서대문구 연희로 32길 51,서대문구,서대문구,연희로
2,서대문형무소역사관,37.572442,126.959541,서울특별시 서대문구 통일로251,서대문구,서대문구,주의!!서울특별시 서대문구 통일로251
3,서울교육박물관,37.581031,126.983694,서울특별시 종로구 북촌로 5길 48,서울특별시특별시교육청,서울특별시특별시교육청,북촌로
4,서울상상나라,37.549458,127.086261,서울특별시 광진구 능동로 216,서울특별시특별시,서울특별시특별시,능동로
...,...,...,...,...,...,...,...
154,포스코미술관,37.505831,127.056088,서울특별시 강남구 대치4동 892 포스코센터 내,㈜포스코,㈜포스코,대치4동
155,학아재미술관,NaN,NaN,서울특별시 종로구 돈화문로 98,김로이,김로이,돈화문로
156,한미사진미술관,37.516608,127.113970,"서울특별시 송파구 위례성대로 14 한미타워 19, 20층",가현문화재단,가현문화재단,위례성대로
157,헬로우뮤지움,37.507358,127.040312,서울특별시 성동구 금호로 72,김효정\n(김이삭),김효정\n(김이삭),금호로


특이사항

구가 안써있거나,

바로 끝나버리는 경우(서울시 oo구 oo로)

얘네만 추가 라벨링 해주면 될듯(str.contains('주의!!')해서)


### 도서관

In [1092]:
lib_df

,도서관명,위도,경도,시군구명,소재지도로명주소,운영기관명
103,대방어린이도서관,37.503024,126.925222,동작구,서울특별시 동작구 대방동길 55,서울특별시 동작구청
105,동작어린이도서관,37.509204,126.943434,동작구,서울특별시 동작구 장승배기로16길 98,서울특별시 동작구청
106,동작상도국주도서관,37.506822,126.950996,동작구,서울특별시 동작구 매봉로 37,서울특별시 동작구청
202,넓은들작은도서관,37.511385,126.841418,양천구,서울특별시 양천구 목동중앙남로 16나길 55,서울특별시 양천구청
203,고맙습니다 작은도서관,37.515237,126.854773,양천구,서울특별시 양천구 중앙로 209,서울특별시 양천구청
...,...,...,...,...,...,...
3299,서울특별시교육청도봉도서관(디지털자료실),37.652746,127.012966,서울특별시 도봉구,서울특별시 도봉구 도봉구 삼양로 556(쌍문동),NaN
3300,도봉문화정보도서관(종합자료실),37.644581,127.043883,서울특별시 도봉구,서울특별시 도봉구 도봉구 덕릉로 315 (창동),NaN
3301,도봉문화정보도서관(어린이열람실),37.644581,127.043883,서울특별시 도봉구,서울특별시 도봉구 도봉구 덕릉로 315 (창동),NaN
3302,도봉어린이문화정보도서관(영 유아),37.659007,127.049470,서울특별시 도봉구,서울특별시 도봉구 도봉구 노해로69길 151,NaN


In [1093]:
len(lib_df.소재지도로명주소)

523

In [1094]:
lib_df.소재지도로명주소.iloc[0]

'서울특별시 동작구 대방동길 55'

In [1095]:
lib_df['동'] = 0
for idx in range(len(lib_df.소재지도로명주소)):
    if re.findall('구 (.+)', lib_df.소재지도로명주소.iloc[idx]):        

        if re.findall('(.+?) ', re.findall('구 (.+)', lib_df.소재지도로명주소.iloc[idx])[0]):
            lib_df['동'].iloc[idx] = re.findall('(.+?) ', re.findall('구 (.+)', lib_df.소재지도로명주소.iloc[idx])[0])[0]
            print(re.findall('(.+?) ', re.findall('구 (.+)', lib_df.소재지도로명주소.iloc[idx])[0])[0])
            
        else:
            lib_df['동'].iloc[idx] = '주의!!' + lib_df.소재지도로명주소.iloc[idx]            
            print('주의!!' + lib_df.소재지도로명주소.iloc[idx])
    else:
        lib_df['동'].iloc[idx] = '주의!!' + lib_df.소재지도로명주소.iloc[idx]
        print('주의!!' + lib_df.소재지도로명주소.iloc[idx])

대방동길
장승배기로16길
매봉로
목동중앙남로
중앙로
목동중앙남로
목동중앙남로
지양로17길
목동중앙남로
목동중앙남로
목동중앙남로
목동중앙남로
목동중앙본로
화곡로4길
화곡로4길
화곡로4길
중앙로52길
목동서로
목동중앙남로
오목로5길
목동남로
주의!!서울특별시 동작구 양녕로23길27
주의!!서울특별시 동작구 사당로20길132
동작대로29길
양녕로22바길
솔밭로
노원로34길
한글비석로
월계로
동일로243길
노원로34길
노원로1나길
마들로
마들로
마들로
마들로
석계로
초안산로5길
동일로186길
노원로1길
노원로
공릉로55길
중계로
한글비석로
노원로
덕릉로82길
동일로
상계로
덕릉로
한글비석로
노해길
동일로227길
한글비석로
노원로
동일로
동일로
덕릉로
마들로3길
화랑로
동일로
구로동로26길
디지털로27다길
구로중앙로27나길
경인로
구로동로13길
개봉로16길
고척로27바길
오리로
신도림로
구로중앙로40길
경인로19길
동남로
도신로33길
당산로
영등포로25길
주의!!서울특별시 영등포구 국제금융로124
주의!!서울특별시 영등포구 영신로200
주의!!서울특별시 영등포구 영등포로64길15
신길로61길
국회대로
여의대로
선유로
문래로20길
주의!!서울특별시 영등포구 도신로27
선유로
주의!!서울특별시 영등포구 대림로197
주의!!서울특별시 영등포구 대림로23길25
주의!!서울특별시 영등포구 디지털로441
주의!!서울특별시 영등포구 여의대방로43길10
대방천로
주의!!서울특별시 영등포구 도림로264
주의!!서울특별시 영등포구 신길로42길1
주의!!서울특별시 영등포구 신길로41라길13-8
주의!!서울특별시 영등포구 영등포로84길24-5
주의!!서울특별시영등포구 선유로47길30(4층)
주의!!서울특별시 영등포구 영등포로11길12
주의!!서울특별시 영등포구 도림로141가길16
자하문로36길
종로58가길
명륜길
삼봉로
북촌로
삼일대로30길
자하문로13길
숭인동길
자하문로
평창문화로
통일로14길
송월길
종로35가길
이화장길
혜화로
창신길
지봉로
용답중앙
송정
광나루로
뚝섬로
왕십리로
뚝섬로

<ipython-input-1095-d081f7f14f42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lib_df['동'] = 0
C:\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-1095-d081f7f14f42>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lib_df['동'].iloc[idx] = re.findall('(.+?) ', re.fin

In [1096]:
lib_df

,도서관명,위도,경도,시군구명,소재지도로명주소,운영기관명,동
103,대방어린이도서관,37.503024,126.925222,동작구,서울특별시 동작구 대방동길 55,서울특별시 동작구청,대방동길
105,동작어린이도서관,37.509204,126.943434,동작구,서울특별시 동작구 장승배기로16길 98,서울특별시 동작구청,장승배기로16길
106,동작상도국주도서관,37.506822,126.950996,동작구,서울특별시 동작구 매봉로 37,서울특별시 동작구청,매봉로
202,넓은들작은도서관,37.511385,126.841418,양천구,서울특별시 양천구 목동중앙남로 16나길 55,서울특별시 양천구청,목동중앙남로
203,고맙습니다 작은도서관,37.515237,126.854773,양천구,서울특별시 양천구 중앙로 209,서울특별시 양천구청,중앙로
...,...,...,...,...,...,...,...
3299,서울특별시교육청도봉도서관(디지털자료실),37.652746,127.012966,서울특별시 도봉구,서울특별시 도봉구 도봉구 삼양로 556(쌍문동),NaN,도봉구
3300,도봉문화정보도서관(종합자료실),37.644581,127.043883,서울특별시 도봉구,서울특별시 도봉구 도봉구 덕릉로 315 (창동),NaN,도봉구
3301,도봉문화정보도서관(어린이열람실),37.644581,127.043883,서울특별시 도봉구,서울특별시 도봉구 도봉구 덕릉로 315 (창동),NaN,도봉구
3302,도봉어린이문화정보도서관(영 유아),37.659007,127.049470,서울특별시 도봉구,서울특별시 도봉구 도봉구 노해로69길 151,NaN,도봉구


중간정리:
일단, 주소가 있는 곳들은 구 다음에 나오는 00동이나 00로를 가지고 라벨링하게 했는데,
00로 로 끝나버리는 곳들은 주의에러가 뜨는 경우, 
데이터 내에서 띄어쓰기가 잘못되어있는경우,
도봉구 도봉구처럼 데이터자체가 문제가 있는 경우등이 있어서
전처리를 더 해줘야 한다.

하지만 이 전처리가 다 된다고 해도
00로 만 가지고 ~동으로 매칭할 수 있을지 알아봐야 함.

요약: 확실한 전처리 더 필요, 도로명주소와 동매칭 탐구필요 

# 2번은 행정동코드사용과 s9s의 도움으로 안해도 될듯

### 3번을 위한 haversine코드 

# 구단위 매칭

In [1097]:
close_gu = {'구' : [], '공원' : []}
for idx, park in tqdm(enumerate(park_df.공원명)):
    start = (float(park_df.loc[idx]['위도']), float(park_df.loc[idx]['경도']))
    distance_dic = {}
    for idx, gu in enumerate(gu_loc_df.시군구명_한글):
        goal = (gu_loc_df.loc[idx]['위도'].mean(), gu_loc_df.loc[idx]['경도'].mean())
        distance_dic[gu] = float(haversine(start, goal))
    closest = sorted((value, key) for key, value in distance_dic.items())[0][1]
    close_gu['구'].append(closest)
    close_gu['공원'].append(park)
mapping = pd.DataFrame(close_gu)
mapping = mapping[['공원','구']]
mapping

,공원,구
0,남산도시자연공원,중구
1,길동생태공원,강동구
2,서울대공원,서초구
3,서울숲,성동구
4,월드컵공원,마포구
...,...,...
127,용마도시자연공원(사가정공원),중랑구
128,문화비축기지,마포구
129,경춘선숲길,노원구
130,율현공원,송파구


# 함수짜기

loc_df에는 구/행정동의 위도경도가 담겨있어야 하고

data_df에는 데이터와 위도경도가,

name에는 data의 이름(도서관명/지하철역명)이 지정되야 한다.

'시군구명_한글'은 loc_df에 들어가는 구/행정동의 열 이름이 들어가면 된다

# 구단위

In [1098]:
def gu_mapping(loc_df, data_df, name):
    close_gu = {'구' : [], '{}'.format(name) : []}
    for idx, data in tqdm(enumerate(data_df['{}'.format(name)])):
        start = (float(data_df.iloc[idx]['위도']), float(data_df.iloc[idx]['경도']))
        distance_dic = {}
        for idx, gu in enumerate(loc_df.시군구명_한글):#loc_df가 바뀌면 같이 바꿔주어야 함
            goal = (loc_df.iloc[idx]['위도'].mean(), loc_df.iloc[idx]['경도'].mean())
            distance_dic[gu] = float(haversine(start, goal))
        closest = sorted((value, key) for key, value in distance_dic.items())[0][1]
        close_gu['구'].append(closest)
        close_gu['{}'.format(name)].append(data)
    mapping = pd.DataFrame(close_gu)
    mapping = mapping[['{}'.format(name),'구']]
    return mapping

In [1099]:
bus_gu_map = gu_mapping(gu_loc_df, bus_df, '정류장명')
bus_gu_map

,정류장명,구
0,종로2가사거리,중구
1,창경궁.서울대학교병원,중구
2,명륜3가.성대입구,종로구
3,종로2가.삼일교,중구
4,혜화동로터리.여운형활동터,종로구
...,...,...
11275,우성아파트,강동구
11276,우성아파트,강동구
11277,조일약국,강동구
11278,성내시장,강동구


In [1100]:
bus_gu_map.groupby(['구']).count()

,정류장명
구,
강남구,365
강동구,373
강북구,336
강서구,394
관악구,254
광진구,335
구로구,525
금천구,476
노원구,401


In [1101]:
park_gu_map = gu_mapping(gu_loc_df, park_df, '공원명')
park_gu_map

,공원명,구
0,남산도시자연공원,중구
1,길동생태공원,강동구
2,서울대공원,서초구
3,서울숲,성동구
4,월드컵공원,마포구
...,...,...
127,용마도시자연공원(사가정공원),중랑구
128,문화비축기지,마포구
129,경춘선숲길,노원구
130,율현공원,송파구


In [1102]:
park_gu_map.groupby(['구']).count()

,공원명
구,
강남구,5
강동구,7
강북구,3
강서구,6
관악구,2
광진구,2
구로구,5
금천구,4
노원구,4


In [1103]:
museum_gu_map = gu_mapping(gu_loc_df, museum_df, '시설명')
museum_gu_map

,시설명,구
0,둘리뮤지엄,강북구
1,서대문자연사박물관,서대문구
2,서대문형무소역사관,서대문구
3,서울교육박물관,종로구
4,서울상상나라,광진구
...,...,...
154,포스코미술관,강남구
155,학아재미술관,도봉구
156,한미사진미술관,송파구
157,헬로우뮤지움,강남구


In [1104]:
museum_gu_map.groupby(['구']).count()

,시설명
구,
강남구,5
강동구,1
강북구,3
강서구,3
관악구,2
광진구,3
구로구,1
노원구,2
도봉구,33


In [1105]:
lib_gu_map = gu_mapping(gu_loc_df, lib_df, '도서관명')
lib_gu_map

,도서관명,구
0,대방어린이도서관,동작구
1,동작어린이도서관,동작구
2,동작상도국주도서관,동작구
3,넓은들작은도서관,양천구
4,고맙습니다 작은도서관,양천구
...,...,...
518,서울특별시교육청도봉도서관(디지털자료실),강북구
519,도봉문화정보도서관(종합자료실),강북구
520,도봉문화정보도서관(어린이열람실),강북구
521,도봉어린이문화정보도서관(영 유아),도봉구


In [1106]:
lib_gu_map.groupby(['구']).count()

,도서관명
구,
강남구,15
강동구,5
강북구,13
강서구,19
관악구,34
광진구,1
구로구,12
금천구,31
노원구,24


# 동단위

In [1107]:
def dong_mapping(loc_df, data_df, name):
    close_dong = {'법정동코드' : [], '{}'.format(name) : []}
    for idx, data in tqdm(enumerate(data_df['{}'.format(name)])):
        start = (float(data_df.iloc[idx]['위도']), float(data_df.iloc[idx]['경도']))
        distance_dic = {}
        for idx, dong in enumerate(loc_df.법정동코드):#loc_df가 바뀌면 같이 바꿔주어야 함
            goal = (loc_df.iloc[idx]['위도'].mean(), loc_df.iloc[idx]['경도'].mean())
            distance_dic[dong] = float(haversine(start, goal))
        closest = sorted((value, key) for key, value in distance_dic.items())[0][1]
        close_dong['법정동코드'].append(closest)
        close_dong['{}'.format(name)].append(data)
    mapping = pd.DataFrame(close_dong)
    mapping = mapping[['{}'.format(name),'법정동코드']]
    return mapping

In [1108]:
dong_loc_df

,법정동코드,위도,경도
3546,1111010100,37.587111,126.969069
3547,1111010200,37.583911,126.968354
3548,1111010300,37.584381,126.971489
3549,1111010400,37.582416,126.971670
3550,1111010500,37.580363,126.972065
...,...,...,...
4008,1174010600,37.531157,127.142891
4009,1174010700,37.552497,127.133010
4010,1174010800,37.531338,127.129174
4011,1174010900,37.543676,127.131748


In [1109]:
bus_dong_mapping  = dong_mapping(dong_loc_df, bus_df, '정류장명')
bus_dong_mapping.to_pickle('버스법정동매칭.pkl')
bus_dong_mapping

,정류장명,법정동코드
0,종로2가사거리,1111013800
1,창경궁.서울대학교병원,1111016600
2,명륜3가.성대입구,1111017200
3,종로2가.삼일교,1111013500
4,혜화동로터리.여운형활동터,1111016900
...,...,...
11275,우성아파트,1174010700
11276,우성아파트,1174010700
11277,조일약국,1171010300
11278,성내시장,1174010800


In [1110]:
bus_dong_map = bus_dong_mapping.groupby(['법정동코드']).count().reset_index()
bus_dong_map

,법정동코드,정류장명
0,1111010100,4
1,1111010300,2
2,1111010400,3
3,1111010500,2
4,1111010600,1
...,...,...
425,1174010600,50
426,1174010700,37
427,1174010800,47
428,1174010900,35


In [955]:
park_dong_mapping = dong_mapping(dong_loc_df, park_df, '공원명')
park_dong_mapping.to_pickle('공원법정동매칭.pkl')
park_dong_mapping

,공원명,법정동코드
0,남산도시자연공원,1114014200
1,길동생태공원,1174010500
2,서울대공원,1165010400
3,서울숲,1120011400
4,월드컵공원,1144012700
...,...,...
127,용마도시자연공원(사가정공원),1126010100
128,문화비축기지,1144012700
129,경춘선숲길,1135010300
130,율현공원,1168011300


In [963]:
park_dong_map = park_dong_mapping.groupby(['법정동코드']).count().reset_index()
park_dong_map

,법정동코드,공원명
0,1111010100,2
1,1111010900,1
2,1111011300,1
3,1111011900,1
4,1111012100,1
...,...,...
100,1171011400,1
101,1174010100,2
102,1174010200,2
103,1174010500,2


In [957]:
museum_dong_mapping = dong_mapping(dong_loc_df, museum_df, '시설명')
museum_dong_mapping.to_pickle('박물관법정동매칭.pkl')
museum_dong_mapping

,시설명,법정동코드
0,둘리뮤지엄,1132010500
1,서대문자연사박물관,1141011700
2,서대문형무소역사관,1111018000
3,서울교육박물관,1111014300
4,서울상상나라,1121510200
...,...,...
154,포스코미술관,1168010600
155,학아재미술관,1111010100
156,한미사진미술관,1171011100
157,헬로우뮤지움,1168010100


In [961]:
museum_dong_map = museum_dong_mapping.groupby(['법정동코드']).count().reset_index()
museum_dong_map

,법정동코드,시설명
0,1111010100,33
1,1111010600,1
2,1111011100,1
3,1111011800,1
4,1111012100,1
...,...,...
87,1168011400,1
88,1168011800,1
89,1171010200,3
90,1171011100,4


In [959]:
lib_dong_mapping = dong_mapping(dong_loc_df, lib_df, '도서관명')
lib_dong_mapping.to_pickle('도서관법정동매칭.pkl')
lib_dong_mapping

,도서관명,법정동코드
0,대방어린이도서관,1159010800
1,동작어린이도서관,1159010100
2,동작상도국주도서관,1159010300
3,넓은들작은도서관,1153010600
4,고맙습니다 작은도서관,1147010100
...,...,...
518,서울특별시교육청도봉도서관(디지털자료실),1130510400
519,도봉문화정보도서관(종합자료실),1132010700
520,도봉문화정보도서관(어린이열람실),1132010700
521,도봉어린이문화정보도서관(영 유아),1132010600


In [962]:
lib_dong_map = lib_dong_mapping.groupby(['법정동코드']).count().reset_index()
lib_dong_map

,법정동코드,도서관명
0,1111010100,72
1,1111010300,1
2,1111010800,1
3,1111011900,1
4,1111013300,1
...,...,...
175,1171011400,1
176,1174010700,1
177,1174010800,1
178,1174010900,2


### 정리:
복지시설 데이터 중,

버스정류소/공원/박물관+미술관/도서관 데이터를 구해서 전처리 후 위도경도와 주소데이터를 구했다.

체육시설 데이터는 하은이 데이터반출 기다리고 있고,

지하철데이터는 아직 좋은 자료를 구하지 못했다.


행정동과 복지시설을 매칭하는 방법을 3가지 생각해서 시도했다.

1번, 2번, 3번중에

1번은 전처리 계속 하다보면 유의미한 결과가 나올 듯 하지만,

'버스'를 커버하지 못한다는점과 도로명주소-행정동 매칭이 필요하다는 점이 문제다.

2번은 행정동별 더 좋은 위도경도 데이터를 구할 수 있으면 그렇게 하면 되고 안되면 다른방법 알아봐야함.

3번은 구별로 라벨링은 성공했지만 동별로 가야하니 2번과 같은 문제.

와우 소연이가 자료 지하철 구했대!

### 정리2차:
복지시설 대부분의 데이터(지하철/공원/박물관/도서관)를 

에스나인에스라는 행정동-법정동-주소 매칭해주는 서비스 제공 회사에서 라벨링 해주기로 했다.

haversine을 활용하면 가장 가까운 곳 중심으로만 매칭되기에, 

정확한 동 구분은 힘들었는데(동 위치 좌표에 대해서도 개인블로그 데이터라서 신뢰도도 높지 않음)

주소를 가지고 제대로 행정동매칭 해준다면 데이터의 신뢰도가 크게 상승된다.

회사에서 데이터 주는것을 기다리며, 시험삼아 haversine 함수를 이용해 

전체 데이터를 구별/동별 매칭을 시켰다.
    

### 추가:
동 좌표 출처에는 행정 법정동 코드라고 있어서 뭔소린가 했는데 비교해보니 그냥 법정동 코드였다..
행정동으로 다시 라벨링 해주기.

### 문제:
행정동 코드로 매칭시키면 267개밖에 나오지 않는다. 이유는 행정동 좌표가 중복적으로 있기 때문.
법정동 코드로 매칭시키면 679개가 나오니, 법정-행정 매칭 데이터프레임을 참고해서 그걸 행정동에 배정하는게 더 정확할 듯 하다.

In [1111]:
# with open('./버스행정동매칭.pkl','rb') as up:
#     bus_dong_mapping = pickle.load(up)

In [1112]:
# bus_dong_map = bus_dong_mapping.groupby('행정동코드').count().reset_index()
# bus_dong_map.columns = ['법정동코드','정류장명']
# bus_dong_map

# 아래부터 보면 됨!

# 받은데이터(지하철,공원,도서관,박물관)+만든데이터(버스) 행정동별 코드 중심으로 데이터프레임 만들기

In [1113]:
subway_dong = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\S9S동매칭/지하철.csv',encoding='utf-8',header=None)
park_dong = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\S9S동매칭/공원.csv',encoding='utf-8',header=None)
museum_dong = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\S9S동매칭/박물관.csv',encoding='utf-8',header=None)
library_dong = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\S9S동매칭/도서관.csv',encoding='utf-8',header=None)

In [1114]:
df_match = dong_df.merge(bus_dong_map, on='법정동코드')
dong_bus = df_match[['시군구명','읍면동명','행정동코드','정류장명']].groupby(['시군구명','읍면동명','행정동코드']).sum().reset_index()
dong_bus.columns = ['시군구명','행정동명','행정동코드','버스정류장']
dong_bus = dong_bus[['행정동코드','버스정류장']]
dong_bus

,행정동코드,버스정류장
0,1168066000,58
1,1168067000,125
2,1168069000,58
3,1168052100,41
4,1168053100,41
...,...,...
420,1126059000,36
421,1126068000,92
422,1126069000,92
423,1126060000,47


In [1115]:
subway_dong

,0,1,2,3,4,5,6,7,8,9
0,1,서울,1,중구,봉래동,서울특별시 중구 봉래동2가 122 서울역(1호선),1.114012e+09,봉래동2가,1.114054e+09,회현동
1,2,시청,1,중구,정동,서울특별시 중구 정동 5-5 시청역(1호선),1.114017e+09,정동,1.114052e+09,소공동
2,3,종각,1,종로구,종로1가,서울특별시 종로구 종로1가 54 종각지하철역사,1.111013e+09,종로1가,1.111062e+09,종로1.2.3.4가동
3,4,종로3가,1,종로구,종로3가,서울특별시 종로3가 10-5 1호선 종로3가역(1호선),1.111016e+09,종로3가,1.111062e+09,종로1.2.3.4가동
4,5,종로5가,1,종로구,종로5가,서울특별시 종로구 종로5가 82-1 1호선 종로5가역,1.111016e+09,종로5가,1.111063e+09,종로5.6가동
...,...,...,...,...,...,...,...,...,...,...
273,274,남한산성입구(성남법원•검찰청),8,성남시,수정구 단대동,경기도 성남시 수정구 단대동 96 남한산성입구역,4.113110e+09,단대동,4.113354e+09,금광2동
274,275,단대오거리,8,성남시,수정구 신흥동,경기도 성남시 수정구 신흥동 2467 단대오거리역,4.113110e+09,신흥동,4.113152e+09,신흥2동
275,276,신흥,8,성남시,수정구 신흥동,경기도 성남시 수정구 신흥동 2467 신흥역,4.113110e+09,신흥동,4.113153e+09,신흥3동
276,277,수진,8,성남시,수정구 수진동,경기도 성남시 수정구 수진동 2205-1 수진역,4.113110e+09,수진동,4.113157e+09,수진1동


In [1116]:
subway_dong.columns=['index','지하철역','호선','시군구명','법정동','주소','전화번호','법정동','전화번호','행정동명']
subway_dong=subway_dong[['지하철역','호선','행정동명']]
subway_dong.drop_duplicates(['지하철역'], keep='first', inplace=True)#다른호선 같은역 제거
subway_dong = subway_dong[['지하철역','행정동명']]
subway_dong

<ipython-input-1116-33c8b6439a50>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subway_dong.drop_duplicates(['지하철역'], keep='first', inplace=True)#다른호선 같은역 제거


,지하철역,행정동명
0,서울,회현동
1,시청,소공동
2,종각,종로1.2.3.4가동
3,종로3가,종로1.2.3.4가동
4,종로5가,종로5.6가동
...,...,...
273,남한산성입구(성남법원•검찰청),금광2동
274,단대오거리,신흥2동
275,신흥,신흥3동
276,수진,수진1동


In [1117]:
subway_dong[subway_dong.행정동명=='충현동']

,지하철역,행정동명
52,충정로(경기대입구),충현동
143,서대문,충현동


In [1118]:
dong_subway = subway_dong.groupby(['행정동명']).count().reset_index()
dong_subway

,행정동명,지하철역
0,가락1동,2
1,가락본동,1
2,가산동,1
3,갈산2동,1
4,거여1동,1
...,...,...
180,화양동,1
181,황학동,1
182,회현동,2
183,효자동,1


### 지하철은 경기도도 포함되어있어서 merge할때 기준열로 삼으면 안된다

In [1119]:
HJD_list = list(dong_df.읍면동명.unique())
HJD_list

[nan,
 '청운효자동',
 '사직동',
 '삼청동',
 '부암동',
 '평창동',
 '무악동',
 '교남동',
 '가회동',
 '종로1.2.3.4가동',
 '종로5.6가동',
 '이화동',
 '혜화동',
 '창신제1동',
 '창신제2동',
 '창신제3동',
 '숭인제1동',
 '숭인제2동',
 '소공동',
 '회현동',
 '명동',
 '필동',
 '장충동',
 '광희동',
 '을지로동',
 '신당동',
 '다산동',
 '약수동',
 '청구동',
 '신당제5동',
 '동화동',
 '황학동',
 '중림동',
 '후암동',
 '용산2가동',
 '남영동',
 '청파동',
 '원효로제1동',
 '원효로제2동',
 '효창동',
 '용문동',
 '한강로동',
 '이촌제1동',
 '이촌제2동',
 '이태원제1동',
 '이태원제2동',
 '한남동',
 '서빙고동',
 '보광동',
 '왕십리제2동',
 '왕십리도선동',
 '마장동',
 '사근동',
 '행당제1동',
 '행당제2동',
 '응봉동',
 '금호1가동',
 '금호2.3가동',
 '금호4가동',
 '옥수동',
 '성수1가제1동',
 '성수1가제2동',
 '성수2가제1동',
 '성수2가제3동',
 '송정동',
 '용답동',
 '화양동',
 '군자동',
 '중곡제1동',
 '중곡제2동',
 '중곡제3동',
 '중곡제4동',
 '능동',
 '광장동',
 '자양제1동',
 '자양제2동',
 '자양제3동',
 '자양제4동',
 '구의제1동',
 '구의제2동',
 '구의제3동',
 '용신동',
 '제기동',
 '전농제1동',
 '전농제2동',
 '답십리제1동',
 '답십리제2동',
 '장안제1동',
 '장안제2동',
 '청량리동',
 '회기동',
 '휘경제1동',
 '휘경제2동',
 '이문제1동',
 '이문제2동',
 '면목제2동',
 '면목제4동',
 '면목제5동',
 '면목본동',
 '면목제7동',
 '면목제3.8동',
 '상봉제1동',
 '상봉제2동',
 '중화제1동',
 '중화제2동',
 '묵제1동',
 '묵제2동',


In [1120]:
park_dong.columns = ['index','공원','위도','경도','시군구명','주소','전처리로','전화번호','법정동','전화번호','행정동명']
park_dong = park_dong[['공원','행정동명']]
park_dong


,공원,행정동명
0,남산도시자연공원,필동
1,길동생태공원,길동
2,서울대공원,문원동
3,서울숲,성수1가제1동
4,월드컵공원,상암동
...,...,...
127,용마도시자연공원(사가정공원),NaN
128,문화비축기지,성산제2동
129,경춘선숲길,NaN
130,율현공원,세곡동


In [1121]:
park_dong[park_dong.행정동명=='신사동']

,공원,행정동명


In [1122]:
for idx, dong in enumerate(park_dong.행정동명):
    print(dong, park_dong.공원.iloc[idx])
        

필동 남산도시자연공원
길동 길동생태공원
문원동 서울대공원
성수1가제1동 서울숲
상암동 월드컵공원
nan 광화문시민열린마당
진관동 진관근린공원(구파발폭포)
영등포동 중마루근린공원
nan 응봉공원
nan 매화근린공원
성수2가제3동 성수근린공원
nan 샛마을근린공원
여의동 여의도근린공원
답십리제1동 간데메근린공원
을지로동 훈련원근린공원
영등포본동 영등포근린공원
신대방제2동 보라매근린공원
능동 어린이대공원
천호제2동 천호근린공원
서빙고동 용산가족공원
천연동 독립근린공원
이화동 낙산근린공원
양재2동 양재시민의숲
대학동 관악산도시자연공원
효창동 효창근린공원
nan 감로천생태공원(관악산)
돈암제1동 개운산근린공원
오류제2동 개웅산근린공원
사직동 경희궁
사당제2동 국립현충원
nan 계남근린공원
고척제2동 고척근린공원
목5동 파리근린공원
가양제2동 허준공원
연희동 궁동근린공원
가양제1동 궁산근린공원
nan 까치산근린공원
nan 꿩고개근린공원
nan 노량진공원
답십리제2동 답십리근린공원
일원본동 대모산도시자연공원
종로1.2.3.4가동 탑골근린공원
nan 갈산근린공원
nan 천마근린공원
nan 도곡근린공원
압구정동 도산근린공원
월곡제2동 청량공원
nan 초안산근린공원
nan 동명근린공원
이화동 마로니에공원
nan 자매근린공원
nan 장지근린공원
nan 명일근린공원
방화제3동 방화근린공원
잠실7동 아시아근린공원
전농제2동 배봉산근린공원
nan 안산도시자연공원
nan 염창근린공원
nan 백련근린공원
오금동 오금근린공원
nan 오동근린공원
nan 봉산도시자연공원
nan 온수도시자연공원
삼성2동 봉은공원
오륜동 올림픽공원
혜화동 와룡근린공원
nan 봉화산근린공원
nan 북악산도시자연공원
nan 북한산국립공원
합정동 외국인묘지공원
nan 불광근린공원
중계4동 불암산도시자연공원
면목제4동 용마폭포공원
노량진제1동 사육신공원
목2동 용왕산근린공원
사직동 사직근린공원
우장산동 우장산근린공원
사당제3동 삼일근린공원
삼청동 삼청근린공원
nan 인왕산도시자연공원
상도제3동 상도근린공원
둔촌제2동 일

In [1123]:
dong_park = park_dong.groupby(['행정동명']).count().reset_index()
dong_park

,행정동명,공원
0,가양제1동,1
1,가양제2동,1
2,고척제2동,1
3,금호2.3가동,1
4,길동,1
...,...,...
68,항동,1
69,혜화동,1
70,화곡본동,1
71,회현동,1


In [1124]:
museum_dong.columns = ['index','박물관/미술관','위도','경도','주소','연락담당','관리담당','전처리로','전화번호','법정동','전화번호','행정동명']
museum_dong = museum_dong[['박물관/미술관','행정동명']]
museum_dong

,박물관/미술관,행정동명
0,둘리뮤지엄,쌍문제1동
1,서대문자연사박물관,연희동
2,서대문형무소역사관,천연동
3,서울교육박물관,삼청동
4,서울상상나라,능동
...,...,...
154,포스코미술관,대치4동
155,학아재미술관,종로1.2.3.4가동
156,한미사진미술관,방이2동
157,헬로우뮤지움,금호1가동


In [1125]:
museum_dong[museum_dong.행정동명=='신사동']

,박물관/미술관,행정동명


In [1126]:
dong_museum = museum_dong.groupby(['행정동명']).count().reset_index()
dong_museum

,행정동명,박물관/미술관
0,가양제2동,1
1,가회동,5
2,개포2동,1
3,고덕제1동,1
4,공릉2동,3
...,...,...
67,화곡제6동,1
68,화양동,1
69,회기동,3
70,회현동,2


In [1127]:
library_dong.columns = ['index','도서관','위도','경도','시군구명','주소','담당부서','전처리로','전화번호','법정동','전화번호','행정동명']
library_dong = library_dong[['도서관','시군구명','행정동명']]
library_dong.loc[library_dong['시군구명']=='서울특별시 도봉구', '시군구명'] = '도봉구'
library_dong

,도서관,시군구명,행정동명
0,대방어린이도서관,동작구,대방동
1,동작어린이도서관,동작구,노량진제2동
2,동작상도국주도서관,동작구,상도제1동
3,넓은들작은도서관,양천구,목3동
4,고맙습니다 작은도서관,양천구,신정3동
...,...,...,...
518,서울특별시교육청도봉도서관(디지털자료실),도봉구,쌍문제1동
519,도봉문화정보도서관(종합자료실),도봉구,창제1동
520,도봉문화정보도서관(어린이열람실),도봉구,창제1동
521,도봉어린이문화정보도서관(영 유아),도봉구,창제4동


In [1128]:
library_dong[library_dong.행정동명=='신사동']

,도서관,시군구명,행정동명
167,신사동작은도서관,강남구,신사동
168,압구정동작은도서관,강남구,신사동
406,신사동 해오름작은도서관,관악구,신사동


In [1129]:
dong_library = library_dong.groupby(['시군구명','행정동명']).count().reset_index()
dong_library 

,시군구명,행정동명,도서관
0,강남구,개포4동,1
1,강남구,논현1동,1
2,강남구,논현2동,1
3,강남구,대치1동,1
4,강남구,대치2동,2
...,...,...,...
281,중랑구,망우본동,2
282,중랑구,면목본동,1
283,중랑구,묵제1동,1
284,중랑구,상봉제2동,1


# 동별 숫자 보기

In [1130]:
data_df = dong_df[['행정동코드','시군구명','읍면동명']].dropna().drop_duplicates(['행정동코드'], keep='first')
data_df.columns = ['행정동코드','시군구명','행정동명']
data_df

,행정동코드,시군구명,행정동명
2,1111051500,종로구,청운효자동
12,1111053000,종로구,사직동
24,1111054000,종로구,삼청동
31,1111055000,종로구,부암동
34,1111056000,종로구,평창동
...,...,...,...
763,1174065000,강동구,성내제2동
764,1174066000,강동구,성내제3동
765,1174068500,강동구,길동
766,1174069000,강동구,둔촌제1동


In [1131]:
dong_bus

,행정동코드,버스정류장
0,1168066000,58
1,1168067000,125
2,1168069000,58
3,1168052100,41
4,1168053100,41
...,...,...
420,1126059000,36
421,1126068000,92
422,1126069000,92
423,1126060000,47


In [1132]:
dong_subway

,행정동명,지하철역
0,가락1동,2
1,가락본동,1
2,가산동,1
3,갈산2동,1
4,거여1동,1
...,...,...
180,화양동,1
181,황학동,1
182,회현동,2
183,효자동,1


In [1133]:
dong_park

,행정동명,공원
0,가양제1동,1
1,가양제2동,1
2,고척제2동,1
3,금호2.3가동,1
4,길동,1
...,...,...
68,항동,1
69,혜화동,1
70,화곡본동,1
71,회현동,1


In [1134]:
dong_museum

,행정동명,박물관/미술관
0,가양제2동,1
1,가회동,5
2,개포2동,1
3,고덕제1동,1
4,공릉2동,3
...,...,...
67,화곡제6동,1
68,화양동,1
69,회기동,3
70,회현동,2


In [1135]:
dong_library

,시군구명,행정동명,도서관
0,강남구,개포4동,1
1,강남구,논현1동,1
2,강남구,논현2동,1
3,강남구,대치1동,1
4,강남구,대치2동,2
...,...,...,...
281,중랑구,망우본동,2
282,중랑구,면목본동,1
283,중랑구,묵제1동,1
284,중랑구,상봉제2동,1


##### 신사동은 두곳이니까 위에서 겹치는 데이터 존재하는지 찾으며 왔고, 도서관만 수정하면 되기에 자료 다시 언급

In [1136]:
library_dong[library_dong.행정동명=='신사동']

,도서관,시군구명,행정동명
167,신사동작은도서관,강남구,신사동
168,압구정동작은도서관,강남구,신사동
406,신사동 해오름작은도서관,관악구,신사동


In [1137]:
d1[d1['행정동명']=='신사동']

,행정동코드,시군구명,행정동명,버스정류장,도서관
332,1162068500,관악구,신사동,1.0,2.0
334,1168051000,강남구,신사동,24.0,2.0


In [1138]:
#중복방지하고, 있는곳만 갯수 채워넣고, 시군구명이 없으면 서울시가 아니니까 제거하기
data_df = data_df.merge(dong_bus, on=['행정동코드'], how='outer').fillna(0).drop_duplicates(['시군구명','행정동명'])
d1 = data_df.merge(dong_library[['행정동명','도서관']], on=['행정동명'], how='outer').fillna(0).drop_duplicates(['시군구명','행정동명'])
d1.iloc[332, 3] = 1.0#관악구 신사동과 강남구 신사동의 경우는 도서관이 각각 1개, 2개이므로 수정
d2 = d1.merge(dong_museum, on=['행정동명'], how='outer').fillna(0).drop_duplicates(['시군구명','행정동명'])
d3 = d2.merge(dong_park, on=['행정동명'], how='outer').fillna(0).drop_duplicates(['시군구명','행정동명'])
d4 = d3.merge(dong_subway, on=['행정동명'], how='outer').fillna(0).drop_duplicates(['시군구명','행정동명'])
df_need = d4[d4.시군구명 != 0]
df_need

,행정동코드,시군구명,행정동명,버스정류장,도서관,박물관/미술관,공원,지하철역
0,1.111052e+09,종로구,청운효자동,27.0,3.0,1.0,0.0,0.0
1,1.111053e+09,종로구,사직동,31.0,0.0,5.0,2.0,2.0
2,1.111054e+09,종로구,삼청동,24.0,1.0,7.0,1.0,0.0
3,1.111055e+09,종로구,부암동,63.0,0.0,5.0,0.0,0.0
4,1.111056e+09,종로구,평창동,51.0,1.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...,...
420,1.174065e+09,강동구,성내제2동,47.0,0.0,0.0,0.0,1.0
421,1.174066e+09,강동구,성내제3동,47.0,1.0,0.0,0.0,1.0
422,1.174068e+09,강동구,길동,34.0,0.0,0.0,1.0,1.0
423,1.174069e+09,강동구,둔촌제1동,50.0,0.0,0.0,0.0,0.0


### 데이터 설명:

버스정류장은 haversine을 통해 법정동 매칭 후 행정동 매칭을 한 데이터.

도서관, 박물관/미술관, 지하철역은 주소를 가지고 s9s 행정동매칭을 한 데이터(데이터 소실 없음)

공원은 주소가 불량해서 132개의 데이터 중 약 73개정도만 얻었는데, 이는 수작업으로 보충해야 할 듯 하지만 

모델을 우선 돌리기 위해 73개만 먼저 데이터 구축함.

# 임시 결론: 

### 공원은 1/2정도밖에 못살려서(주소불량) 하나하나 매칭해줄 수 있지만, 우선 가지고 있는 데이터 기준으로 데이터프레임 작성완료

In [1039]:
df_need.to_pickle("복지시설(대중교통,도서관,박물관,미술관,공원)행정동별매칭.pkl")

# 혹시 몰라서 만드는 '유동인구 데이터 기준 핫한 동네'

유동인구 출처:https://www.bigdatahub.co.kr/product/list.do?menu_id=1000157

In [241]:
#데이터 불러오기
pop1903 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/FLT_SEOUL_03MONTH.csv')
pop1904 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/FLT_SEOUL_04MONTH.csv')
pop1905 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/FLT_SEOUL_05MONTH.csv')
pop1906 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/FLT_SEOUL_06MONTH.csv')
pop1907 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/FLT_SEOUL_07MONTH.csv')
pop1908 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/FLT_SEOUL_08MONTH.csv')
pop1909 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/FLT_SEOUL_09MONTH.csv')
pop1910 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/FLT_SEOUL_10MONTH.csv')
pop1911 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_1911.csv')
pop1912 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_1912.csv')
pop2001 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_2001.csv')
pop2002 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_2002.csv')
pop2003 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_2003.csv')
pop2004 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_2004.csv')
pop2005 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_2005.csv')
pop2006 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_2006.csv')
pop2007 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_2007.csv')
pop2008 = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\SKT데이터허브유동인구/Floating_Population_2008.csv')


In [266]:
population = pd.concat([pop1903,pop1904,pop1905,pop1906,pop1907,pop1908,pop1909,pop1910,pop1911,pop1912,pop2001,pop2002,pop2003,pop2004,pop2005,pop2006,pop2007,pop2008])
population #183메가라서 피클로 저장 후 깃헙에 올리기 불가

,일자,시간(1시간단위),연령대(10세단위),성별,시,군구,유동인구수
0,20190301,0,20,남성,서울,양천구,28720
1,20190301,0,20,남성,서울,종로구,15920
2,20190301,0,20,남성,서울,중랑구,26250
3,20190301,0,20,여성,서울,중랑구,25890
4,20190301,0,30,남성,서울,강동구,34090
...,...,...,...,...,...,...,...
223195,20200831,22,20,남성,서울,서초구,26890
223196,20200831,23,30,남성,서울,성북구,33590
223197,20200831,23,30,여성,서울,동작구,34770
223198,20200831,23,70,여성,서울,금천구,10860


In [270]:
population_df = population.groupby(['일자','군구']).sum()
population_df = population_df['유동인구수'].reset_index()
population_df.to_pickle('구별날짜별유동인구(19년3월부터20년8월까지).pkl')
population_df

,일자,군구,유동인구수
0,20190301,강남구,11667480
1,20190301,강동구,7684500
2,20190301,강북구,5692690
3,20190301,강서구,10313260
4,20190301,관악구,9394200
...,...,...,...
13695,20200831,용산구,5104870
13696,20200831,은평구,7987210
13697,20200831,종로구,5137120
13698,20200831,중구,5528340


# tips...

### 특정 문자열이 그안에 있는 행만 챙기는 법(&쓰면 다중조건가능)
##### df[df['asdf'].str.contains('asdf') & df['zxcv'].str.contains('zxcv')]

# 이것저것 전처리(볼필요X, 나중에 성공하면 위에다 반영할것임)

## 공원

공원번호는 전국 기준인듯하고, 면적은 몇몇개는 re써서 뽑아야할듯. 개원일은 큰 의미있을지는 모르겠다.주요시설에선 valuecount랑 unique본다음에 특수점으로 넣을지 생각하고, 지역이 대부분있는듯하지만 없으면 주소에서 뽑아야할듯. 

In [61]:
park_need = park[['공원명','공원주소','X좌표(WGS84)','Y좌표(WGS84)', '면적']]
park_need.면적 = park_need.면적.astype(str)
park_need

TypeError: string indices must be integers

In [62]:
park_need.head(40)

NameError: name 'park_need' is not defined

In [63]:
for i in range(len(park_need.면적)):
#     print(re.findall('([1-9]*?\.*[1-9]+?)㎡+?m*?²*?', park_need.면적[i]))
    print(re.findall('([0-9]+?\.?[0-9]?)㎡|([0-9]+?\.?[0-9]?)m²|([0-9]+?\.?[0-9]?)㎥', park_need.면적[i]))
    
     

NameError: name 'park_need' is not defined

공원 넓이를 표현하는 방식이 다 다르다
그냥 1000제곱미터(㎡)라고 한곳이 있는 반면, 1000세제곱미터(㎥)로 한곳도, 1000띄어쓰기제곱미터(㎡) 한곳, 1000미터와 제곱(m²) 

배수지용량은 왜나오는데..;